# Notion MCP Agent
## ABB #5 - Session 4

Code authored by: Shaw Talebi

### imports

In [1]:
from agents.mcp.server import MCPServerStdio, MCPServer
from agents import Agent, Runner, gen_trace_id, trace, ModelSettings
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import logging
# Suppress httpx INFO logs to reduce console output
logging.getLogger("httpx").setLevel(logging.WARNING)

In [3]:
# import notion api key
notion_key = os.getenv("NOTION_API_KEY")

### explore MCP server tools

In [4]:
async with MCPServerStdio(
    params={
        "command": "npx",
        "args": ["-y", "@notionhq/notion-mcp-server"],
        "env": {
        "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
        }
    }
) as server:
    tool_list = await server.list_tools()

In [5]:
toolname_list = [tool.name for tool in tool_list]
print(toolname_list)

['API-get-user', 'API-get-users', 'API-get-self', 'API-post-database-query', 'API-post-search', 'API-get-block-children', 'API-patch-block-children', 'API-retrieve-a-block', 'API-update-a-block', 'API-delete-a-block', 'API-retrieve-a-page', 'API-patch-page', 'API-post-page', 'API-create-a-database', 'API-update-a-database', 'API-retrieve-a-database', 'API-retrieve-a-page-property', 'API-retrieve-a-comment', 'API-create-a-comment']


In [6]:
[tool.description for tool in tool_list]

['Retrieve a user\nError Responses:\n400: 400',
 'List all users\nError Responses:\n400: 400',
 "Retrieve your token's bot user",
 'Query a database',
 'Search by title',
 'Retrieve block children',
 'Append block children',
 'Retrieve a block',
 'Update a block',
 'Delete a block',
 'Retrieve a page',
 'Update page properties',
 'Create a page',
 'Create a database',
 'Update a database',
 'Retrieve a database',
 'Retrieve a page property item',
 'Retrieve comments',
 'Create comment']

### create main with MCP server

In [7]:
async def main():
    
    async with MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "@notionhq/notion-mcp-server"],
            "env": {
            "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
            }
        }
    ) as server:
        trace_id = gen_trace_id()
        with trace(workflow_name="Notion Agent Example", trace_id=trace_id):
            print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}\n")
            await run(server)
        
        tools = await server.list_tools()

### create run() with agent

In [8]:
with open('context/example-2/instructions.md', 'r', encoding='utf-8') as file:
    instructions = file.read()

In [9]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Notion Agent",
        model="gpt-4.1-2025-04-14",
        instructions=instructions,
        mcp_servers=[mcp_server],
    )
    ModelSettings.tool_choice = "required"
    
    input_items = []

    print("=== Notion AI Agent ===")
    print("Type 'exit' to end the conversation")

    while True:
        # Get user input
        user_input = input("\nUser: ").strip()
        input_items.append({"content": user_input, "role": "user"})
        
        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye!")
            break
            
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            agent,
            input=input_items,
        )
        
        async for event in result.stream_events():
            # We'll ignore the raw responses event deltas for text
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            elif event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    print(f"\n-- Calling Tool: {event.item.raw_item.name}...")
                elif event.item.type == "tool_call_output_item":
                    print("-- Tool call completed.")
                elif event.item.type == "message_output_item":
                    input_items.append({"content": f"{event.item.raw_item.content[0].text}", "role": "assistant"})
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

### run agent

In [10]:
await main()
# find me posts about AI consulting

View trace: https://platform.openai.com/traces/trace?trace_id=trace_4cfc2ad55e844e178297fd1b6157821f

=== Notion AI Agent ===
Type 'exit' to end the conversation



User:  find me posts about AI consulting



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.
Here are some recent posts about AI consulting in the notes:

1. **How to find (in-demand) AI freelancing service using ChatGPT**
   - Title suggests a deep dive into using ChatGPT as a tool to discover actual market needs for AI consulting/freelancing.

2. **AI won’t make you rich (solving problems will)**
   - This post likely discusses the reality that technical skills alone aren’t enough—solving real client problems is what creates a sustainable consulting business.

3. **Quitting Consulting**
   - A transparent post about leaving consulting to focus on YouTube, along with reasons for the decision.
   - Covers consulting as a career option and when it might make sense to pivot.

Would you like the full content of any of these posts? If so, let me know which one you want to see or if you want a summary/analysis!




User:  Find the upcoming post on motivation and write a draft for it



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.

-- Calling Tool: API-retrieve-a-page...
-- Tool call completed.

-- Calling Tool: API-get-block-children...
-- Tool call completed.
I found an upcoming post titled “Motivation hack: don’t do it for yourself,” scheduled for June 30, 2025. However, there is no body content or additional notes in the draft—just the title.

Given the clear rules against motivational or reflection-based language, and since I can’t fabricate Shaw’s personal experience, here is a draft that follows your guidelines, uses only literal language, and puts the focus on a practical, technical angle based on the prompt.

---

Step 1: Purpose & Audience
- Purpose: Share an actionable mindset shift for staying motivated, grounded in external accountability (not self-focused motivation).
- Audience: Builders, solo entrepreneurs, and technical creators facing dips in motivation.

Step 2: Wireframe
- Hook: A literal, curiosity-driven line based on the 

...


User:  Motivation Hack: Don’t do it for yourself (do it for others). It's easy to procrastinate (or just give up) when you're the only one who's effected by your work.



Agent: Thank you! Here’s how I’ll proceed, following your instructions and guidelines:

---

**Step 1: Purpose & Audience**  
- **Purpose:** Share a practical approach to motivation focused on external accountability (not self-motivation).  
- **Audience:** Builders, creators, and solo entrepreneurs who struggle with motivation.

**Step 2: Wireframe**  
- Hook: Your provided line.  
- Body: Briefly explain why “doing it for others” is more reliable than self-motivation, in a literal and practical way.  
- CTA: Ask for the reader’s practical methods of holding themselves accountable to others.

**Step 3: The Body**  
Most people find it’s much easier to keep going when someone else is counting on them. If your deadline only affects you, it’s easy to push it off—until tomorrow turns into next month. But when you’ve made a commitment to a client, a teammate, or even a small group, procrastination drops. You have built-in accountability.

**Step 4: CTA**  
What’s one method you’ve used to


User:  exit



Goodbye!



User:  Return post ideas about business and entrepreneurship


In [11]:
# # to run in a .py script use
# if __name__ == "__main__":
#     asyncio.run(main())